In [2]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

os.chdir("..")

In [5]:
import yaml
from src.datasets.datasets import CPDDatasets
from src.ensembles.ensembles import EnsembleCPDModel
from src.utils.fix_seeds import fix_seeds
from torch.utils.data import DataLoader

SEED = 42
fix_seeds(SEED)

%load_ext autoreload
%autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Scenario 1: adam pretrain + sgld finetune

In [6]:
model_type = "seq2seq"

experiments_name = "explosion"

path_to_config = "configs/" + "video" + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [ ]:
path_to_models_folder = "saved_models/bce/explosion/layer_norm/train_anomaly_num_155"

ens_bce = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_bce.load_models_list(path_to_models_folder)

In [ ]:
args_config["learning"]["lr"] = 1e-3
args_config["learning"]["temperature"] = 0.001

# for ABAnnealingLR
args_config["learning"]["final_lr"] = 1e-4
args_config["learning"]["gamma"] = 0.5
args_config["learning"]["T_max"] = 50000

core_model = core_models.CombinedVideoRNN(
    input_dim=args_config["model"]["input_size"],
    rnn_hidden_dim=args_config["model"]["hidden_rnn"],
    num_layers=args_config["model"]["rnn_n_layers"],
    rnn_dropout=args_config["model"]["rnn_dropout"],
    dropout=args_config["model"]["dropout"],
    layer_norm=args_config["model"]["layer_norm"],
)

bce_sgld_model = CPDModelCustomNoisyAdam(
    loss_type="bce",
    args=args_config,
    model=core_model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
)

# Scenario 2: sgld pretrain + adam finetune